In [0]:
#Bronze
# 1) BRONZE – land “as-is” JSON into a Delta table, append-only

from pyspark.sql.types import *
from pyspark.sql.functions import current_timestamp, lit

# TODO 1: pick a raw → bronze path naming convention
raw_path    = "/tmp/raw/doc_events/"
bronze_path = "/tmp/bronze/doc_events/"

# TODO 2: decide: rm existing bronze folder on rerun?  (uncomment if yes)
# dbutils.fs.rm(bronze_path, recurse=True)

# TODO 3: define the “as-is” schema for your JSON source
bronze_schema = StructType([
  StructField("event_id",   StringType(), True),
  StructField("user_id",    StringType(), True),
  StructField("doc_id",     StringType(), True),
  StructField("action",     StringType(), True),
  StructField("event_time", StringType(), True),
  StructField("device", StructType([
    StructField("os",     StringType(), True),
    StructField("region", StringType(), True)
  ]), True)
])

# TODO 4: read the raw JSON (hint: spark.read.schema(...).json(...))
df_raw = spark.read \
  .schema(bronze_schema) \
  .json(raw_path + "doc_events.json")

# TODO 5: decide your audit columns; e.g. ingest_ts, batch_id, process_id…
df_bronze = df_raw \
  .withColumn("ingest_ts", current_timestamp()) \
  .withColumn("batch_id",   lit("TODO_YOUR_BATCH_ID"))

# TODO 6: choose your write mode: append vs overwrite; partitioning?
df_bronze.write \
  .format("delta") \
  .mode("append") \
  .save(bronze_path)

# sanity-check
display(spark.read.format("delta").load(bronze_path))